In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [4]:
posts_30K_each = pd.read_csv('/content/drive/MyDrive/TCC II/all_30k_processed_posts.csv')

In [5]:
posts_30K_each

,Unnamed: 0,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message,author,label
0,17518,2020-06-08T16:20:29+0000,167637636622585_3560581307328184,exministro juiz sergio moro detonou conducao g...,o ex-ministro e juiz sergio moro detonou a con...,291,shared_story,https://external.fplu2-1.fna.fbcdn.net/safe_im...,836,92,16,36,11,4,True,left_all_posts,0
1,36123,2018-11-05T14:01:00+0000,219188754789679_2341230762585457,orlandeli,por orlandeli .,1,added_photos,https://scontent.fplu2-1.fna.fbcdn.net/v/t1.64...,13,1,0,0,0,3,True,left_all_posts,0
2,2233,2019-06-02T00:30:19+0000,371913239843447_864924640542302,audio julgamento bolsonaro tribunal militar jo...,audio do julgamento de bolsonaro no tribunal m...,437,added_video,https://scontent.fplu2-1.fna.fbcdn.net/v/t15.5...,201,18,21,1,1,1,True,left_all_posts,0
3,24570,2020-05-20T02:37:30+0000,292074710916413_1524289557744683,conheca plataforma enfrentamento pandemia coro...,conheca a plataforma para o enfrentamento da p...,617,added_video,https://scontent.fplu2-1.fna.fbcdn.net/v/t15.5...,591,7,7,289,19,23,True,left_all_posts,0
4,65984,2020-02-18T21:54:34+0000,127835925882_10157935816100883,forcas opostas fundo manutencao desenvolviment...,forcas opostas : o fundo de manutencao e desen...,85,shared_story,https://external.fplu2-1.fna.fbcdn.net/safe_im...,41,0,1,14,23,2,True,left_all_posts,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61056,3318,2020-07-15T20:01:40+0000,890711084350263_3241472082607473,bolsonaro ainda covid segundo novo exame,bolsonaro ainda esta com covid-19 segundo novo...,5,shared_story,https://external.fplu2-1.fna.fbcdn.net/safe_im...,277,3,0,38,0,3,True,right_all_posts,1
61057,10,2018-03-05T21:23:11+0000,890711084350263_1688888311199199,amanha julgamento hc lula ladrao stj transmiti...,e amanha o julgamento do hc do lula ladrao . s...,69,shared_story,https://external.fplu2-1.fna.fbcdn.net/safe_im...,296,6,5,0,17,10,True,right_all_posts,1
61058,33469,2019-05-07T21:14:44+0000,1965770023473808_2188151608163126,bolsonaro garante recursos colegios militares,bolsonaro garante recursos para colegios milit...,70,added_video,https://scontent.fplu2-1.fna.fbcdn.net/v/t15.1...,404,3,2,0,0,68,True,right_all_posts,1
61059,8196,2019-02-20T02:26:23+0000,550142935316937_847946782203216,presidente jair bolsonaro realidade petistas c...,nosso presidente jair bolsonaro . realidade qu...,214,added_photos,https://scontent.fplu2-1.fna.fbcdn.net/v/t1.64...,440,0,3,0,0,45,True,right_all_posts,1


In [6]:
text = posts_30K_each.pre_processed_message.values
labels = posts_30K_each.label.values

In [7]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [10]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [11]:

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

model.cuda()

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 3

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    model.train()
    
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        train_output.loss.backward()
        optimizer.step()
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    model.eval()

    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        if b_precision != 'nan': val_precision.append(b_precision)
        if b_recall != 'nan': val_recall.append(b_recall)
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

Epoch:  33%|███▎      | 1/3 [06:40<13:21, 400.53s/it]


	 - Train loss: 0.5072
	 - Validation Accuracy: 0.7902
	 - Validation Precision: 0.8383
	 - Validation Recall: 0.7237
	 - Validation Specificity: 0.8552



Epoch:  67%|██████▋   | 2/3 [13:17<06:38, 398.53s/it]


	 - Train loss: 0.3986
	 - Validation Accuracy: 0.7937
	 - Validation Precision: 0.8660
	 - Validation Recall: 0.6999
	 - Validation Specificity: 0.8862



Epoch: 100%|██████████| 3/3 [19:54<00:00, 398.19s/it]


	 - Train loss: 0.3341
	 - Validation Accuracy: 0.8146
	 - Validation Precision: 0.7760
	 - Validation Recall: 0.8902
	 - Validation Specificity: 0.7399



In [17]:
new_sentence = 'bozo'

test_ids = []
test_attention_mask = []

encoding = preprocessing(new_sentence, tokenizer)

test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Direita' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Esquerda'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  bozo
Predicted Class:  Esquerda


In [14]:
model.save_pretrained('/content/drive/MyDrive/TCC II/30K-3-epochs-bert-base-uncased/model-additional-to-test')

In [15]:
train_idx
train = ",".join(map(str, train_idx));
train

'41587,34373,49871,26434,59974,11310,38341,34349,32545,29683,43832,33965,41467,29190,14266,53753,51843,15022,18784,6009,41913,23056,19936,30443,56124,934,44116,41505,13397,7447,20081,22219,56950,10575,10429,26573,20348,16018,53905,5913,22233,40376,12025,31186,13698,44113,2196,39897,49583,46654,47905,17034,54461,28831,51370,27295,26119,50,47695,1958,30400,22361,383,39398,50910,24446,7945,20465,47242,57255,26351,46965,626,55502,59917,38133,46346,31705,23661,44328,32839,26727,39143,33520,8882,16229,13527,7093,31820,25883,44004,56584,15739,22031,13170,46993,36508,7090,3135,10392,7072,48826,2328,6690,37259,2390,7576,26499,32395,22407,41764,38427,33869,33074,13110,55761,8267,60464,57726,53344,7047,11334,34212,29036,25052,34385,36510,11842,44673,4685,16501,27103,42079,271,38619,2459,33522,22440,50673,55763,47911,34259,33526,58330,45992,9024,59914,28233,55446,42731,1991,60212,59462,23725,54712,44228,33309,39403,36790,46719,52921,8143,12805,45441,49133,50769,27427,13824,61025,32271,108,21783,11

In [16]:
val_idx
val = ",".join(map(str, val_idx));
val

'43363,32154,43963,34243,47960,44867,25729,15711,23266,778,60718,5890,16748,7920,43590,28254,24700,18912,14421,56595,28062,53241,44989,1206,55595,40943,54203,33307,6814,31467,15878,27670,56185,27800,59429,2788,30020,40940,30918,1263,45211,37474,30782,57664,292,13321,14066,55265,19848,44772,2500,23194,53486,29448,11695,21005,40538,54938,49119,41890,13230,10183,53294,52680,47806,30393,1843,8224,30406,27465,11139,18628,53196,15149,13643,18720,12073,23124,50617,1291,39608,13461,10687,53373,59356,38321,25707,49414,15817,43542,40070,20852,49121,38540,51903,11522,53632,4669,23270,52215,31385,53508,18130,7604,20829,10577,33395,5873,47528,48019,15977,3478,46040,46815,15472,21822,33302,17411,27489,24198,39902,39720,34096,37635,40743,54411,32818,30589,58206,7682,8335,39640,12131,2064,7420,36716,43293,20147,48587,19555,48731,44697,2245,21048,42756,19422,43660,1867,44208,56559,10727,2509,46861,50949,144,20687,6683,49405,22514,34199,52642,6967,16015,53703,49385,51663,3717,15937,58817,12668,31295,516